<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/multi_agent_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain_openai -q
!pip install langchain_core -q
!pip install pydantic -q
!pip install colab-env -q
import os
import colab_env

In [ ]:
import os
import time
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional
import json

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser

# --- 1. Initialize LLM ---
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [ ]:
# --- 2. Define Pydantic Models for Tool Inputs ---
class AirportInfoInput(BaseModel):
    airport_code: str = Field(..., description="The ICAO or IATA code of the airport (e.g., CYUL for Montreal Trudeau, KJFK for JFK).")

class WeatherForecastInput(BaseModel):
    airport_code: str = Field(..., description="The ICAO or IATA code of the airport for which to get the weather.")
    date: str = Field(..., description="The date for the forecast in YYYY-MM-DD format.")

class RouteOptimizationInput(BaseModel):
    origin_airport_code: str = Field(..., description="The ICAO or IATA code of the departure airport.")
    destination_airport_code: str = Field(..., description="The ICAO or IATA code of the arrival airport.")
    aircraft_type: str = Field(..., description="The type of aircraft (e.g., 'Boeing 737', 'Airbus A320').")
    departure_time: str = Field(..., description="The planned departure time in HH:MM format (24-hour).")
    desired_altitude: Optional[int] = Field(None, description="Optional: Desired cruising altitude in feet.")

# Update AirspaceCheckInput to expect a List of Lists of floats
class AirspaceCheckInput(BaseModel):
    coordinates: List[List[float]] = Field(..., description="A list of [latitude, longitude] pairs representing a segment of the route.")
    altitude: int = Field(..., description="The altitude in feet for which to check airspace restrictions.")

# --- 3. Define Custom Tools for the Flight Planning Agent ---

@tool("get_airport_info", args_schema=AirportInfoInput)
def get_airport_info(airport_code: str) -> Dict[str, Any]:
    """
    Fetches basic information about a given airport, including its name and active runways.
    """
    print(f"\n--- Tool Call: get_airport_info({airport_code}) ---")
    # Simulate API call to an airport database
    airport_data = {
        "CYUL": {"name": "Montreal-Pierre Elliott Trudeau International Airport", "runways": ["06L/24R", "06R/24L", "10/28"]},
        "KJFK": {"name": "John F. Kennedy International Airport", "runways": ["04L/22R", "04R/22L", "13L/31R", "13R/31L"]},
        "KLAX": {"name": "Los Angeles International Airport", "runways": ["06L/24R", "06R/24L", "07L/25R", "07R/25L"]},
        "EHAM": {"name": "Amsterdam Airport Schiphol", "runways": ["18C/36C", "18L/36R", "18R/36L", "04/22", "06/24", "09/27"]},
    }
    info = airport_data.get(airport_code.upper(), {"name": "Unknown Airport", "runways": []})
    print(f"--- Tool Result: {json.dumps(info, indent=2)} ---")
    return info

@tool("get_weather_forecast", args_schema=WeatherForecastInput)
def get_weather_forecast(airport_code: str, date: str) -> Dict[str, Any]:
    """
    Provides a simplified weather forecast for a specific airport on a given date.
    """
    print(f"\n--- Tool Call: get_weather_forecast({airport_code}, {date}) ---")
    # Simulate fetching weather data (could be METARs/TAFs in real world)
    current_date = datetime.now().strftime("%Y-%m-%d")
    if date == current_date:
        if airport_code.upper() == "CYUL":
            weather = {"conditions": "Light rain, moderate winds from West (15-20 knots), visibility 8 km, temperature 10°C.", "wind_direction_deg": 270, "wind_speed_knots": 18, "turbulence_risk": "low"}
        elif airport_code.upper() == "KJFK":
            weather = {"conditions": "Clear skies, light winds from North (5-10 knots), visibility 10 km, temperature 15°C.", "wind_direction_deg": 360, "wind_speed_knots": 8, "turbulence_risk": "very low"}
        else:
            weather = {"conditions": "Partly cloudy, light winds, temperature 20°C.", "turbulence_risk": "low"}
    else:
        # Simulate a future forecast
        weather = {"conditions": "Expected clear, light winds, no significant weather.", "turbulence_risk": "very low"}

    print(f"--- Tool Result: {json.dumps(weather, indent=2)} ---")
    return weather

@tool("optimize_flight_route", args_schema=RouteOptimizationInput)
def optimize_flight_route(
    origin_airport_code: str,
    destination_airport_code: str,
    aircraft_type: str,
    departure_time: str,
    desired_altitude: Optional[int] = None
) -> Dict[str, Any]:
    """
    Calculates an optimized flight route, fuel estimate, and estimated flight time
    considering origin, destination, aircraft type, and current weather.
    It simulates finding the most efficient path.
    """
    print(f"\n--- Tool Call: optimize_flight_route({origin_airport_code}, {destination_airport_code}, {aircraft_type}, {departure_time}, altitude={desired_altitude}) ---")

    # Simulate route calculation logic
    # In a real system, this would involve complex algorithms, possibly external APIs
    # for flight planning software (e.g., Jeppesen, Lido).

    distance_km = 600  # Example distance for CYUL-KJFK
    avg_speed_kmh = 800  # Example average cruising speed

    if "Boeing 737" in aircraft_type:
        fuel_burn_per_hr = 2500  # kg/hr
        time_factor = 1.0
        route_details = ["CYUL departure", "FL350", "via JRO", "KJFK arrival"]
    elif "Airbus A320" in aircraft_type:
        fuel_burn_per_hr = 2300  # kg/hr
        time_factor = 0.98
        route_details = ["CYUL departure", "FL360", "via ALB", "KJFK arrival"]
    else:
        fuel_burn_per_hr = 2000
        time_factor = 1.0
        route_details = ["Direct", "FL340"]

    estimated_flight_hours = (distance_km / avg_speed_kmh) * time_factor
    estimated_flight_minutes = int(estimated_flight_hours * 60)
    estimated_fuel_kg = fuel_burn_per_hr * estimated_flight_hours

    # Add a simple weather impact simulation (e.g., headwind increases time/fuel)
    # This would integrate with a real weather agent's output.
    # Removed the direct call to get_weather_forecast to avoid CallbackManager issues.
    # We will simulate a basic weather check here instead.
    weather_impact_factor = 1.0
    # Simple simulation: if origin is CYUL or KJFK, simulate some potential headwind impact
    if origin_airport_code.upper() in ["CYUL", "KJFK"]:
         # In a real scenario, you'd use weather data obtained by the agent
         # before calling this tool, or pass it as an argument.
         # Simulating a condition that might cause a slight increase:
         simulated_wind_speed = 15 # knots
         simulated_wind_direction = 280 # degrees
         if simulated_wind_speed > 10 and simulated_wind_direction > 180 and simulated_wind_direction < 360:
             weather_impact_factor = 1.05 # Simulate slight increase for headwinds


    estimated_flight_minutes = int(estimated_flight_minutes * weather_impact_factor)
    estimated_fuel_kg = int(estimated_fuel_kg * weather_impact_factor)

    optimized_plan = {
        "route": route_details,
        "estimated_flight_time_minutes": estimated_flight_minutes,
        "estimated_fuel_burn_kg": estimated_fuel_kg,
        "recommended_altitude_feet": desired_altitude if desired_altitude else (35000 if "Boeing" in aircraft_type else 36000),
        "remarks": "Route optimized for fuel efficiency and directness. Check latest NOTAMs."
    }
    print(f"--- Tool Result: {json.dumps(optimized_plan, indent=2)} ---")
    return optimized_plan

@tool("check_airspace_restrictions", args_schema=AirspaceCheckInput)
def check_airspace_restrictions(coordinates: List[List[float]], altitude: int) -> Dict[str, Any]:
    """
    Checks for any temporary or permanent airspace restrictions (e.g., prohibited areas,
    restricted zones, temporary flight restrictions (TFRs)) along a given set of coordinates at a specified altitude.
    Returns details of any restrictions found.
    """
    print(f"\n--- Tool Call: check_airspace_restrictions({coordinates}, {altitude}) ---")
    # Simulate complex airspace database query
    restrictions_found = []
    # Example: a temporary restriction near JFK for a specific altitude
    if any(coord[0] > 40.0 and coord[0] < 41.0 and coord[1] < -73.0 and coord[1] > -75.0 for coord in coordinates) and altitude < 10000:
        restrictions_found.append({
            "type": "TFR",
            "description": "Temporary Flight Restriction near New York Class B airspace due to special event. Below 10,000 ft.",
            "effective_from": (datetime.now() - timedelta(hours=1)).isoformat(),
            "effective_to": (datetime.now() + timedelta(hours=3)).isoformat(),
            "impact": "Requires re-routing below 10,000ft, or coordination with ATC if transiting."
        })
    if altitude > 45000:
         restrictions_found.append({
            "type": "High Altitude Restriction",
            "description": "Max civilian altitude typically FL450 without special clearance.",
            "impact": "Consider lower altitude or special clearance."
        })

    result = {"restrictions": restrictions_found, "status": "OK" if not restrictions_found else "Restrictions Found"}
    print(f"--- Tool Result: {json.dumps(result, indent=2)} ---")
    return result

# --- 4. Assemble the Tools ---
tools = [
    get_airport_info,
    get_weather_forecast,
    optimize_flight_route,
    check_airspace_restrictions
]

# --- 5. Create the Agent Prompt ---
current_date_str = datetime.now().strftime("%Y-%m-%d")
current_time_str = datetime.now().strftime("%H:%M:%S")

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a sophisticated OpenAI flight planning agent. "
     "Your goal is to assist in planning efficient, safe, and compliant flights. "
     "Use the provided tools to gather information and generate optimal flight plans.\n"
     f"Current date: {current_date_str}, Current time: {current_time_str} EDT.\n"
     "Remember to consider weather, air traffic, airport conditions, and aircraft performance for every plan."
     "When asked to plan a flight, ensure you provide details like estimated flight time, fuel burn, and a recommended route."
     "If specific details like aircraft type or desired altitude are missing, ask for them."
     ),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"), # Essential for agents to store their thinking process
])

# --- 6. Create the Agent and Executor ---
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- 7. Run the Agent with a Sample Query ---
if __name__ == "__main__":
    print("--- Starting OpenAI Flight Planning Agent ---")

    # Example 1: Basic flight plan request
    print("\n\n--- Query 1: Plan a flight from Montreal to New York for a Boeing 737. ---")
    result1 = agent_executor.invoke({
        "input": "Plan a flight from Montreal (CYUL) to New York (KJFK) for a Boeing 737."
    })
    print("\n--- Agent's Final Answer for Query 1 ---")
    print(result1["output"])

    # Example 2: More detailed request, including a specific date and asking for weather
    print("\n\n--- Query 2: What's the weather like in Amsterdam (EHAM) for tomorrow? Then plan a flight from London (EGLL, though tool only supports EHAM) to Amsterdam for an Airbus A320 for tomorrow, what would be the optimal route and fuel? ---")
    tomorrow_date = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")
    result2 = agent_executor.invoke({
        "input": f"What's the weather like in Amsterdam (EHAM) for tomorrow? Then plan a flight from London (EGLL, I know only EHAM is supported for now) to Amsterdam for an Airbus A320 for tomorrow, what would be the optimal route and fuel?"
    })
    print("\n--- Agent's Final Answer for Query 2 ---")
    print(result2["output"])

    # Example 3: Asking for airport info directly
    print("\n\n--- Query 3: Tell me about Los Angeles International Airport (KLAX). ---")
    result3 = agent_executor.invoke({
        "input": "Tell me about Los Angeles International Airport (KLAX)."
    })
    print("\n--- Agent's Final Answer for Query 3 ---")
    print(result3["output"])

    # Example 4: A query that might trigger airspace check (simulated coordinates)
    print("\n\n--- Query 4: Are there any airspace restrictions at 8000 feet around these coordinates: [[40.7128, -74.0060], [40.7580, -73.9855]]? ---")
    result4 = agent_executor.invoke({
        "input": "Are there any airspace restrictions at 8000 feet around these coordinates: [[40.7128, -74.0060], [40.7580, -73.9855]]?"
    })
    print("\n--- Agent's Final Answer for Query 4 ---")
    print(result4["output"])

--- Starting OpenAI Flight Planning Agent ---


--- Query 1: Plan a flight from Montreal to New York for a Boeing 737. ---


> Entering new AgentExecutor chain...

Invoking: `get_weather_forecast` with `{'airport_code': 'CYUL', 'date': '2025-05-28'}`
responded: To plan the flight from Montreal (CYUL) to New York (KJFK) for a Boeing 737, I need to gather some information about the current weather conditions at both airports and any potential airspace restrictions. I will also calculate the optimized flight route, estimated flight time, and fuel requirements.

Let's start by gathering the necessary information.


--- Tool Call: get_weather_forecast(CYUL, 2025-05-28) ---
--- Tool Result: {
  "conditions": "Light rain, moderate winds from West (15-20 knots), visibility 8 km, temperature 10\u00b0C.",
  "wind_direction_deg": 270,
  "wind_speed_knots": 18,
  "turbulence_risk": "low"
} ---
{'conditions': 'Light rain, moderate winds from West (15-20 knots), visibility 8 km, temperature 10°C.', 